# Import Library

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import re
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from langdetect import detect

import time
from IPython.display import display, clear_output, Markdown
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from collections import defaultdict
import numpy as np
from tqdm import tqdm
from ragas.llms import LangchainLLMWrapper
from ragas.evaluation import evaluate
from ragas.metrics import Faithfulness, FactualCorrectness, LLMContextRecall
from ragas import EvaluationDataset
from ragas import RunConfig

/home/mshidayat66/anaconda3/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Set seed untuk NumPy, PyTorch, dan Python Random
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# Indexing

In [3]:
# Fungsi preprocessing teks
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text)  # Normalisasi spasi
    text = re.sub(r"[()!*\-?]", "", text)
    return text.strip()

In [4]:
# Load CSV
df = pd.read_csv("./dataset.csv", encoding="utf-8")

print(df)

                                              question  \
0                                   What is Carbuncle?   
1                               What Causes Carbuncle?   
2    What are the risk factors and frequency for ca...   
3        What are the signs and Symptoms of Carbuncle?   
4                             How is carbuncle tested?   
..                                                 ...   
298  What are the Treatment Options for Leprosy (Ha...   
299         What else can Leprosy (Hansen Disease) be?   
300      What to expect with Leprosy (Hansen Disease)?   
301  What are the Possible Complications When Diagn...   
302     How can Leprosy (Hansen Disease) be prevented?   

                                                answer  \
0    A carbuncle is a cluster of boils, called furu...   
1    Carbuncles, which are infections in the hair f...   
2    Carbuncles, while common, occur most often in ...   
3    A person with a carbuncle, which is a kind of ...   
4    If your 

In [5]:
# Preprocessing
df = df.fillna("")  # Hindari NaN
df["question"] = df["question"].apply(preprocess_text)
df["answer"] = df["answer"].apply(preprocess_text)
df["source"] = df["source"].apply(preprocess_text)
df["focus_area"] = df["focus_area"].apply(preprocess_text)

# Konversi ke list
questions = df["question"].tolist()
answers = df["answer"].tolist()
sources = df["source"].tolist()
focus_areas = df["focus_area"].tolist()

In [6]:
# Inisialisasi Model dan Database
client = QdrantClient("http://34.69.51.79:6333")
llama = Ollama(model="llama3.1:8b", temperature=0.0, mirostat_tau=4.0, mirostat_eta=0.65, timeout=None)
model = SentenceTransformer('BAAI/bge-m3', device='cuda' if torch.cuda.is_available() else 'cpu')
embeddings = model.encode(answers)

/tmp/ipykernel_3088/1202965713.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llama = Ollama(model="llama3.1:8b", temperature=0.0, mirostat_tau=4.0, mirostat_eta=0.65, timeout=None)


In [7]:
# Membuat collection dan menginput data ke Qdrant
client.recreate_collection(
    collection_name='Skin Diseases',
    vectors_config=VectorParams(
        size=len(embeddings[0]),
        distance=Distance.COSINE
    ),
    quantization_config=models.ScalarQuantization(
        scalar=models.ScalarQuantizationConfig(
            type=models.ScalarType.INT8
        )
    )
)

# Menyiapkan data points
points = []
for i in range(len(embeddings)):
    points.append(
        PointStruct(
            id=i,
            vector=embeddings[i],
            payload={
                "question"   : questions[i]    if i < len(questions)    else "",
                "answer"     : answers[i]      if i < len(answers)      else "",
                "source"     : sources[i]      if i < len(sources)      else "",
                "focus_area" : focus_areas[i]  if i < len(focus_areas)  else "",
            }
        )
    )

# Mengirim data dalam batch
batch_size = 50
for i in range(0, len(points), batch_size):
    batch_points = points[i:i+batch_size]
    client.upsert(
        collection_name='Skin Diseases',
        wait=True,
        points=batch_points
    )
    print(f'Uploaded batch {i // batch_size + 1}')

/tmp/ipykernel_3088/2789142302.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Uploaded batch 1
Uploaded batch 2
Uploaded batch 3
Uploaded batch 4
Uploaded batch 5
Uploaded batch 6
Uploaded batch 7


In [8]:
# Fungsi deteksi dan translate ke English (jika perlu)
def detect_and_translate(text):
    try:
        detected_lang = detect(text)
        if detected_lang == "en":
            return text
        else:
            translation_prompt = f"""
            Translate the following sentence from {detected_lang} to English. Do not add explanation, just translate.

            Original ({detected_lang}): {text}
            English:
            """
            translated = llama.invoke(translation_prompt)
            return translated.strip()
    except Exception as e:
        return text

In [9]:
def search(query):
    # Vector Query
    query_vector = model.encode(query).tolist()

    results = client.search(
        collection_name='Skin Diseases',
        query_vector=query_vector,
        limit=5,
        with_payload=True,
        score_threshold=0.4
    )

    # Mengurutkan berdasarkan score
    sorted_result = sorted(results, key=lambda x: x.score, reverse=True)
    
    # Ambil payload dengan penanganan jika key tidak tersedia
    final_results = []
    for res in sorted_result:
        payload = res.payload
        questions = payload.get('question', '')
        answer = payload.get('answer', '')
        source = payload.get('source', '')
        focus_area = payload.get('focus_area', '')
        combined = f"Q: {questions}\nA: {answer}\nSource: {source}\nFocus Area: {focus_area}".strip()
        final_results.append(combined)

    return final_results

# Contoh Query
query = "Apa itu karbunkel atau Carbuncle?"
query = detect_and_translate(query)
results = search(query)

# Tampilkan hasil pencarian
for result in results:
    print(result)

Q: what is carbuncle
A: a carbuncle is a cluster of boils, called furuncles, that are connected to each other. this occurs when an infection starts in one or more hair follicles and spreads to the skin and deeper layers of tissue surrounding these follicles. carbuncles usually appear as red, tender, and swollen lumps with several oozing openings or pusfilled bumps on the surface. they often come with general symptoms of illness, and swollen glands may also occur in the nearby area. while carbuncles can appear anywhere on the body where there is hair, they are most commonly found on areas with thicker skin like the back of the neck, back, and thighs. if an infected hair follicle, known as folliculitis, doesn’t get treated, it can turn into a boil, or furuncle. when these boils cluster together, they create a carbuncle. you can have one carbuncle or several at the same time.
Source: nih national library of medicine
Focus Area: carbuncle atau karbunkel
Q: what causes carbuncle
A: carbuncl

# Implementasi Retrieval Augmented Generation

In [10]:
# Prompt Template
prompt_template = PromptTemplate(
    input_variables=["context", "query", "tone"],
    template=""" 
    You are a Healthcare chatbot named C-Skin. Please respond to each user question using the tone: {tone}.
    
    Instructions for answering disease-related questions:
    - Answer using only the information provided in {context}.
    - If the question is related to a disease, provide accurate, concise, and complete information based on the context.
    - If no relevant information is found in the context, do not speculate or fabricate answers. Instead, reply with: "This is all the information I have."
    - Avoid using complex or unfamiliar medical terms. Keep the language simple and understandable for general users.
    - Always begin your response with: "Thank you for consulting with C-Skin."
    - Always end your response with: "I hope this information helps, and I wish you a speedy recovery, and thank you."
    - Do not use any external knowledge beyond the provided context.
    - Do not mention that you are limited by context — just provide what you can from it.
    - You must not infer or invent facts. Only rephrase or summarize what exists explicitly in the context.

    Only use the following context to answer the user's question:
    {context}

    User Question:
    {query}

    Answer in English only.
    """
)


# Mendefinisikan LLM Chain
llm_chain = LLMChain(
    llm=llama,
    prompt=prompt_template
)

# Fungsi untuk mengenerate response
def generate(context, query, tone="professional and friendly"):
    context_text = "\n\n".join([f"Doc {i+1}:\n{ctx.strip()}" for i, ctx in enumerate(context)])
    response = llm_chain.run(context=context_text, query=query, tone=tone)
    return response

# Question Query
query = "Saya memiliki benjolan di ketiak, ketika dipegang lembut tapi sakit dan terlihat seperti ada nanahnya, kira-kira penyakit apa ya?"
query = detect_and_translate(query)
results = search(query)
response = generate(results, query)

# Menampilkan Response
output = ""
for line in response.split("\n"):
    output += line
    clear_output(wait=True)
    display(Markdown(output))
    time.sleep(0.1)

Thank you for consulting with C-Skin.Based on the information provided, it seems that your symptoms are consistent with a carbuncle. A carbuncle is a type of skin infection that can appear as a growing tender bump on the body, often accompanied by pus-filled bumps on top. It's common to find carbuncles in areas with hair, such as the armpits.The symptoms you described, including pain when touched gently and the presence of pus, are also consistent with a carbuncle. However, it's essential to consult a healthcare professional for an accurate diagnosis and proper treatment.I hope this information helps, and I wish you a speedy recovery, and thank you.

# Evaluasi

In [11]:
# Daftar pertanyaan evaluasi
eval_questions = [
    "Apa itu karbunkel atau Carbuncle?",
    "Apa itu jerawat atau Acne Vulgaris?",
    "apa itu vitiligo?",
    "Apa itu panu atau Tinea Versicolor?",
    "Apa itu kusta atau Leprosy ?"
]
eval_answers = [
    "Karbunkel atau Carbuncle adalah infeksi kulit yang serius, ditandai dengan kumpulan bisul (furunkel) yang saling terhubung di bawah permukaan kulit. Infeksi ini biasanya melibatkan beberapa folikel rambut beserta jaringan di sekitarnya, sehingga menimbulkan benjolan yang merah, bengkak, dan terasa nyeri. Karbunkel umumnya lebih parah dibandingkan bisul tunggal, lebih berisiko meninggalkan bekas luka, dan sering disertai gejala sistemik seperti demam serta rasa tidak enak badan.",
    "Jerawat atau Acne Vulgaris adalah gangguan kulit yang disebabkan oleh penyumbatan pori-pori akibat penumpukan minyak (sebum), sel kulit mati, dan kotoran. Jerawat dapat menyebabkan peradangan dan infeksi, dan umumnya muncul di wajah, leher, punggung, dan dada.",
    "Vitiligo adalah kondisi kulit di mana pigmen atau warna kulit hilang, menyebabkan munculnya bercak-bercak putih atau pucat di kulit. Kondisi ini terjadi karena sel-sel yang memproduksi melanin, zat yang memberi warna pada kulit, berhenti bekerja. Vitiligo bukan penyakit menular dan tidak berbahaya.",
    "Panu, atau dikenal sebagai Tinea Versicolor atau Pityriasis Versicolor, adalah infeksi jamur pada kulit yang menyebabkan munculnya bercak-bercak kecil dengan perubahan warna. Bercak tersebut dapat tampak lebih terang atau lebih gelap dibandingkan dengan kulit di sekitarnya.",
    "Kusta (lepra) atau Leprosy adalah penyakit infeksi kronis yang disebabkan oleh bakteri *Mycobacterium leprae*. Kusta terutama menyerang kulit, saraf tepi, saluran pernapasan atas, dan mata, dan dapat menyebabkan kerusakan permanen jika tidak ditangani dengan tepat."
]

# Wrap the Ollama model with LangchainLLMWrapper for RAGAS evaluation
evaluator_llm = LangchainLLMWrapper(llama)

# Menyiapkan data evaluasi
dataset = []
for query_item, reference in tqdm(zip(eval_questions, eval_answers), total=len(eval_questions), desc="Generating responses"):
    query_item = detect_and_translate(query_item)
    reference = detect_and_translate(reference)
    
    relevant_docs = search(query_item)
    response_text = generate(relevant_docs, query_item)

    dataset.append({
        "user_input": query_item,
        "retrieved_contexts": relevant_docs,
        "response": response_text,
        "reference": reference,
    })

# Evaluasi langsung seluruh dataset
print("\nEvaluating entire dataset:")
eval_dataset = EvaluationDataset.from_list(dataset)

run_config = RunConfig(timeout=None)

results = evaluate(
    dataset=eval_dataset,
    metrics=[Faithfulness(), LLMContextRecall(llm=evaluator_llm)],
    llm=evaluator_llm,
    embeddings=model,
    batch_size=10,
    run_config=run_config
)

Generating responses: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [37:39<00:00, 451.86s/it]


Evaluating entire dataset:


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Batch 1/1:   0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
# Gabungkan semua skor dari hasil evaluasi
metric_scores = defaultdict(list)

for score_dict in results.scores:
    for metric_name, score in score_dict.items():
        metric_scores[metric_name].append(score)

# Hitung dan tampilkan rata-rata skor
print("\n=== Rata-rata Skor Evaluasi ===")
for metric_name, scores in metric_scores.items():
    average_score = np.nanmean(scores)
    print(f"{metric_name}: {average_score:.4f}")


=== Rata-rata Skor Evaluasi ===
faithfulness: 0.9429
context_recall: 0.9600


In [13]:
print(metric_scores)

defaultdict(<class 'list'>, {'faithfulness': [1.0, 1.0, 1.0, 1.0, 0.7142857142857143], 'context_recall': [0.8, 1.0, 1.0, 1.0, 1.0]})


In [14]:
# Daftar pertanyaan evaluasi
eval_questions = [
    "Apa itu karbunkel atau Carbuncle?",
    "Apa itu jerawat atau Acne Vulgaris?",
    "Apa itu kanker kulit atau Skin Cancer?",
    "Apa itu panu atau Tinea Versicolor?",
    "Apa itu kusta atau Leprosy ?"
]
eval_answers = [
    "Karbunkel adalah infeksi kulit yang serius, ditandai dengan kumpulan bisul (furunkel) yang saling terhubung di bawah permukaan kulit. Infeksi ini biasanya melibatkan beberapa folikel rambut beserta jaringan di sekitarnya, sehingga menimbulkan benjolan yang merah, bengkak, dan terasa nyeri. Karbunkel umumnya lebih parah dibandingkan bisul tunggal, lebih berisiko meninggalkan bekas luka, dan sering disertai gejala sistemik seperti demam serta rasa tidak enak badan.",
    "Jerawat atau Acne Vulgaris adalah gangguan kulit yang disebabkan oleh penyumbatan pori-pori akibat penumpukan minyak (sebum), sel kulit mati, dan kotoran. Jerawat dapat menyebabkan peradangan dan infeksi, dan umumnya muncul di wajah, leher, punggung, dan dada.",
    "Kanker kulit atau Skin Cancer adalah kondisi pertumbuhan sel kulit yang abnormal dan tidak terkendali, yang dapat berkembang menjadi tumor ganas. Kanker kulit biasanya terjadi di area tubuh yang sering terpapar sinar matahari, seperti wajah, leher, dan tangan, tetapi juga dapat muncul di bagian tubuh lain yang jarang terkena sinar matahari.",
    "Panu, atau dikenal sebagai Tinea Versicolor atau Pityriasis Versicolor, adalah infeksi jamur pada kulit yang menyebabkan munculnya bercak-bercak kecil dengan perubahan warna. Bercak tersebut dapat tampak lebih terang atau lebih gelap dibandingkan dengan kulit di sekitarnya.",
    "Kusta (lepra) atau Leprosy adalah penyakit infeksi kronis yang disebabkan oleh bakteri *Mycobacterium leprae*. Kusta terutama menyerang kulit, saraf tepi, saluran pernapasan atas, dan mata, dan dapat menyebabkan kerusakan permanen jika tidak ditangani dengan tepat."
]


# Wrap the Ollama model with LangchainLLMWrapper for RAGAS evaluation
evaluator_llm = LangchainLLMWrapper(llama)

# Menyiapkan data evaluasi
dataset = []
for query_item, reference in tqdm(zip(eval_questions, eval_answers), total=len(eval_questions), desc="Generating responses"):
    query_item = detect_and_translate(query_item)
    reference = detect_and_translate(reference)
    
    relevant_docs = search(query_item)
    response_text = generate(relevant_docs, query_item)

    dataset.append({
        "user_input": query_item,
        "retrieved_contexts": relevant_docs,
        "response": response_text,
        "reference": reference,
    })

# Evaluasi langsung seluruh dataset
print("\nEvaluating entire dataset:")
eval_dataset = EvaluationDataset.from_list(dataset)

run_config = RunConfig(timeout=None)

results = evaluate(
    dataset=eval_dataset,
    metrics=[FactualCorrectness(mode='f1', atomicity="low", coverage="low"), FactualCorrectness(mode='recall', atomicity="low", coverage="low"), FactualCorrectness(mode='precision', atomicity="low", coverage="low")],
    llm=evaluator_llm,
    embeddings=model,
    batch_size=10,
    run_config=run_config
)

Generating responses: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [38:37<00:00, 463.59s/it]


Evaluating entire dataset:


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Batch 1/2:   0%|          | 0/10 [00:00<?, ?it/s]

In [15]:
# Gabungkan semua skor dari hasil evaluasi
metric_scores = defaultdict(list)

for score_dict in results.scores:
    for metric_name, score in score_dict.items():
        metric_scores[metric_name].append(score)

# Hitung dan tampilkan rata-rata skor
print("\n=== Rata-rata Skor Evaluasi ===")
for metric_name, scores in metric_scores.items():
    average_score = np.nanmean(scores)
    print(f"{metric_name}: {average_score:.4f}")


=== Rata-rata Skor Evaluasi ===
factual_correctness(mode=f1): 0.7080
factual_correctness(mode=recall): 0.7640
factual_correctness(mode=precision): 0.4720


In [16]:
print(metric_scores)

defaultdict(<class 'list'>, {'factual_correctness(mode=f1)': [0.62, 0.62, 0.7, 0.89, 0.71], 'factual_correctness(mode=recall)': [0.67, 0.8, 0.75, 0.6, 1.0], 'factual_correctness(mode=precision)': [0.57, 0.5, 0.4, 0.5, 0.39]})


# Integrasi Chainlit